In [24]:
import pandas as pd
import numpy as np
from utils.utils import easyPro, easyConnect
import tushare as ts
from sqlalchemy import create_engine, select, Table, Column, String, MetaData
from sqlalchemy.orm import sessionmaker
def stock2index(stock_codes: list[str], start_date: str, end_date: str) -> pd.DataFrame:
    # 股票代码对应到指数代码
    # 需要：数据库中有申万行业分类sw_category
    engine = easyConnect()
    Session = sessionmaker(bind=engine)
    session = Session()
    metadata = MetaData()
    your_table = Table(
        'sw_category', metadata,
        Column('ts_code', String(50), primary_key=True),
        Column('l3_code', String(50))
    )
    try:
        # 创建查询语句
        stmt = select(
            your_table.c.ts_code,
            your_table.c.l3_code
        ).where(
            your_table.c.ts_code.in_(stock_codes)
        )

        # 执行查询
        result = session.execute(stmt)

        # 构建字典
        s2i = {row.ts_code: row.l3_code for row in result}
        # 确保所有目标值都有条目（可选）
        for item in stock_codes:
            if item not in s2i:
                s2i[item] = None  # 或设置默认值

        print(s2i)

    finally:
        session.close()  # 确保关闭会话
        session.close()  # 确保关闭会话

In [25]:
stock2index(['000001.SZ', '600519.SH', '300750.SZ'],'20200501','20250501')

{'600519.SH': '851251.SI', '300750.SZ': '857371.SI', '000001.SZ': None}


In [9]:
pro=easyPro()
pro.sw_daily(ts_code='851425.SI')

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,float_mv,total_mv
0,851425.SI,20250813,纸包装,1994.90,1973.59,2029.20,2006.80,14.07,0.71,31344.0,230806.0,41.91,1.97,5108957.0,10732700.0
1,851425.SI,20250812,纸包装,2002.57,1976.91,2030.32,1992.73,-9.69,-0.48,35181.0,262148.0,42.58,1.97,5183516.0,11308588.0
2,851425.SI,20250811,纸包装,1956.46,1934.92,2019.16,2002.43,53.24,2.73,38021.0,260184.0,42.79,1.98,5208734.0,11371098.0
3,851425.SI,20250808,纸包装,1949.00,1922.41,1969.03,1949.19,-0.07,0.00,27931.0,184069.0,41.57,1.93,5070254.0,11111322.0
4,851425.SI,20250807,纸包装,1938.49,1922.26,1972.18,1949.26,17.11,0.89,30052.0,202964.0,41.56,1.93,5070437.0,11125728.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,851425.SI,20090309,纸包装,1105.28,1035.95,1124.12,1067.91,-35.09,-3.18,1436.0,11132.0,30.60,3.00,321629.0,889829.0
3996,851425.SI,20090306,纸包装,1083.88,1066.71,1125.83,1103.00,1.02,0.09,963.0,10350.0,32.42,3.10,332196.0,931176.0
3997,851425.SI,20090305,纸包装,1079.05,1049.82,1117.22,1101.98,34.93,3.27,1567.0,14719.0,32.15,3.09,331890.0,926852.0
3998,851425.SI,20090304,纸包装,1007.60,1006.39,1071.06,1067.05,59.45,5.90,1204.0,9866.0,30.79,3.00,321369.0,892545.0


In [6]:
from utils.utils import easyPro, easyConnect
import pandas as pd
engine = easyConnect()
target_list = ['000001.SZ', '600519.SH', '300750.SZ']  # 替换为实际需要的值
# 构建查询语句
query = """
SELECT ts_code, l1_code, l2_code, l3_code
FROM sw_category
WHERE ts_code IN ({})
""".format(','.join(['%s']*len(target_list)))

# 读取数据到 DataFrame
df = pd.read_sql(query, engine, params=tuple(target_list))

# 创建字典
s2i_l1 = df.set_index('ts_code')['l1_code'].to_dict()
s2i_l2 = df.set_index('ts_code')['l2_code'].to_dict()
s2i_l3 = df.set_index('ts_code')['l3_code'].to_dict()

# 添加缺失项
for code in target_list:
    if code not in s2i_l1:
        s2i_l1[code] = None
    if code not in s2i_l2:
        s2i_l2[code] = None
    if code not in s2i_l3:
        s2i_l3[code] = None


TypeError: not all arguments converted during string formatting

In [3]:
s2i_l1

{'600519.SH': '801120.SI', '000001.SZ': '801780.SI', '300750.SZ': '801730.SI'}

In [4]:
s2i_l2

{'600519.SH': '801125.SI', '000001.SZ': '801783.SI', '300750.SZ': '801737.SI'}

In [5]:
s2i_l3

{'600519.SH': '851251.SI', '000001.SZ': '857831.SI', '300750.SZ': '857371.SI'}

In [40]:
df = pro.index_member_all(ts_code='000001.SZ')

In [41]:
df

,l1_code,l1_name,l2_code,l2_name,l3_code,l3_name,ts_code,name,in_date,out_date,is_new
0,801780.SI,银行,801783.SI,股份制银行Ⅱ,857831.SI,股份制银行Ⅲ,000001.SZ,平安银行,19910403,None,Y


In [26]:
pro=easyPro()
l1 = pro.index_classify(level='l1', src='sw2021', fields='index_code,is_pub')

In [27]:
l1

,index_code,is_pub
0,801010.SI,1
1,801030.SI,1
2,801040.SI,1
3,801050.SI,1
4,801080.SI,1
5,801880.SI,1
6,801110.SI,1
7,801120.SI,1
8,801130.SI,1
9,801140.SI,1


In [14]:
l2['is_pub'].astype('int').sum()

np.int64(124)

In [33]:
if l1[l1['index_code'] == '801010.SI']['is_pub'].iloc[0] == '1':
    print("1")

1


In [35]:
def stock2index(stock_codes: list[str]) -> pd.DataFrame:
    # 股票代码对应到指数代码
    # 需要：数据库中有申万行业分类sw_category
    engine=easyConnect()
    pro = easyPro()
    query = """
    SELECT ts_code, l1_code, l2_code, l3_code
    FROM sw_category
    WHERE ts_code IN ({})
    """.format(','.join(['%s']*len(stock_codes)))

    df = pd.read_sql(query, con=engine, params=tuple(stock_codes))
    s2i_l1 = df.set_index('ts_code')['l1_code'].to_dict()
    s2i_l2 = df.set_index('ts_code')['l2_code'].to_dict()
    s2i_l3 = df.set_index('ts_code')['l3_code'].to_dict()

    # 添加缺失项
    for code in stock_codes:
        if code not in s2i_l1:
            s2i_l1[code] = None
        if code not in s2i_l2:
            s2i_l2[code] = None
        if code not in s2i_l3:
            s2i_l3[code] = None
    s2i = {}
    # 逐项检查指数是否有行情 已知一级行业全有行情
    if_l3 = pro.index_classify(level='l3', src='sw2021', fields='index_code,is_pub')
    if_l2 = pro.index_classify(level='l2', src='sw2021', fields='index_code,is_pub')
    for stock, index_l3 in s2i_l3.items():
        if index_l3 is not None:
            if if_l3[if_l3['index_code'] == index_l3]['is_pub'].iloc[0] is True:
                s2i[stock] = index_l3
            else:
                index_l2 = s2i_l2[stock]
                if if_l2[if_l2['index_code'] == index_l2]['is_pub'].iloc[0] is True:
                    s2i[stock] = index_l2
                else:
                    s2i[stock] = s2i_l1[stock] # index_l1
    return s2i

In [36]:
ret = stock2index(['000001.SZ','600598.SH', '300021.SZ'])
ret

{'600598.SH': '801010.SI', '300021.SZ': '801010.SI', '000001.SZ': '801780.SI'}

In [1]:
from utils.utils import easyPro, easyConnect
pro = easyPro()

08-20 10:51:33 | INFO    | utils.logger_config:_setup_logger | 📝 日志系统启动成功
08-20 10:51:33 | INFO    | utils.logger_config:_setup_logger | 📁 主日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250820-1.log
08-20 10:51:33 | INFO    | utils.logger_config:_setup_logger | 🔴 错误日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250820-1_error.log
08-20 10:51:33 | INFO    | utils.logger_config:_setup_logger | 📂 项目根目录: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos


In [2]:
conn = easyConnect()

In [4]:
import pandas as pd
sql = pd.read_sql('select * from sw_category', con=conn)

In [11]:
sql['l1_code'].unique().shape

(31,)

In [14]:
temp = pd.read_csv('../assets/250818TO250819.csv')

In [15]:
temp.to_sql('temp_data', con=conn, if_exists='replace', index=False)

10823

In [16]:
temp['total_mv'].notnull().sum()

np.int64(10823)